# Step 1: Preparation
## 1 ETL
### 1.1 读取数据

In [1]:
# 导入modules
import pandas as pd
import numpy as np
import os

In [26]:
from IPython.core.display import display
os.listdir('../data')
df_raw = pd.read_csv('../data/data.csv', encoding='ISO-8859-1', dtype={'CustomerID': str, 'InvoiceID': str})
df_raw["InvoiceDate"] = pd.to_datetime(df_raw["InvoiceDate"])

print("DataFrame Shape: ", df_raw.shape)

DataFrame Shape:  (541909, 8)


In [27]:
df_raw.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


这里对每个变量做一些解释：
*InvoiceNo*: 每一笔订单的单号.
*StockCode*: 订单中的一种商品编号
*Description*: 对这种商品的描述
*Quantity*: 在这一订单中这种商品的数量
*InvoiceDate*: 每笔订单的发生时间
*UnitPrice*: 该商品的单价
*CustomerID*: 购买者的用户ID
*Country*: 购买用户的国家

### 1.2 数据清洗
#### 1.2.1 数据缺失

In [28]:
# 缺失数据数量统计
df_raw.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [29]:
# 缺失数据占比
pd.DataFrame(df_raw.isnull().sum()) / df_raw.shape[0] * 100

,0
InvoiceNo,0.000000
StockCode,0.000000
Description,0.268311
Quantity,0.000000
InvoiceDate,0.000000
UnitPrice,0.000000
CustomerID,24.926694
Country,0.000000


Note: 观测到这里缺失的顾客ID信息占比近25%！思考这缺失的原因：可能是在该网站上买东西时不需要强制登陆。但我们可以假设这25%不是由那75%的用户产生的（实际中一般有账号的用户买东西时都会登陆），这样不同的用户之间可以看作是独立的，我们就直接删去这些缺失数据！

In [30]:
df_raw.dropna(axis=0, inplace=True)

In [31]:
df_raw.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

删去空的CustomerID的数据后，已经没有空值了。

#### 1.2.2 异常数据

In [37]:
# 先看InvoiceNo
df_raw["InvoiceNo"].value_counts().iloc[0:20]

576339    542
579196    533
580727    529
578270    442
573576    435
567656    421
567183    399
575607    377
571441    364
570488    353
572552    352
568346    335
547063    294
569246    285
562031    277
554098    264
543040    259
570672    259
569897    239
572103    223
Name: InvoiceNo, dtype: int64

In [38]:
# 观察InvoiceNo的长度
df_raw["InvoiceNo"].apply(lambda l: len(l)).value_counts()
# 发现还有7位的，接下来仔细看一下

6    397924
7      8905
Name: InvoiceNo, dtype: int64

In [40]:
long_invoice_no = df_raw[df_raw["InvoiceNo"].apply(lambda l: len(l)) != 6]
long_invoice_no.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


Note: 发现C开头的订单都是退货的信息，这不是我们目前想要深入了解的，目前我们先专注于购买订单，因此把这里的8905条退货数据删除。

In [76]:
df_cleaned = df_raw.loc[df_raw["InvoiceNo"].apply(lambda l: l[0] != 'C')]

df_cleaned.shape[0]

397924

In [77]:
# 接下来看StockCode
df_cleaned["StockCode"].value_counts()
# 并没有看出什么不对，再看一下长度

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
          ... 
84613C       1
21414        1
84550        1
90037B       1
90202A       1
Name: StockCode, Length: 3665, dtype: int64

In [78]:
df_cleaned["StockCode"].apply(lambda l: len(l)).value_counts()
# 发现有多种长度的StockCode, 最长的长达12

5     363119
6      32960
4       1103
7        291
1        290
2        133
3         16
12        12
Name: StockCode, dtype: int64

In [79]:
StockCode_length = df_cleaned["StockCode"].apply(lambda l: len(l))
display(df_cleaned[StockCode_length == 1].head(3))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274,United Kingdom
6798,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723,United Kingdom


In [80]:
# 观测特殊StockCode的Description情况
for i in [1,2,3,4,12]:
    print(df_cleaned[StockCode_length == i].Description.value_counts())

Manual    290
Name: Description, dtype: int64
CARRIAGE    133
Name: Description, dtype: int64
DOTCOM POSTAGE    16
Name: Description, dtype: int64
POSTAGE                       1099
PADS TO MATCH ALL CUSHIONS       4
Name: Description, dtype: int64
Bank Charges    12
Name: Description, dtype: int64


Note: 发现当StockCode为1、2、3、4、12时Description分别为Manual、CARRIAGE、DOTCOM POSTAGE、POSTAGE、Bank Charges。这些运费等附加费用不是我们的主要盈利来源，所以我们先不需要这些信息，一并删除。

In [81]:
df_cleaned = df_cleaned[df_cleaned["StockCode"].apply(lambda l: len(l) in [5,6,7])]

In [86]:
# 再看CustomerID
df_cleaned.CustomerID.value_counts()

17841    7838
14911    5591
14096    5095
12748    4580
14606    2697
         ... 
16323       1
13302       1
12814       1
13106       1
17752       1
Name: CustomerID, Length: 4335, dtype: int64

In [89]:
df_cleaned.CustomerID.apply(lambda l: len(l)).value_counts()
# 看上去都正常

5    396370
Name: CustomerID, dtype: int64

In [103]:
# 再看Quantity
display(df_cleaned.Quantity.describe())
# 发现极大值是异常点

count    396370.000000
mean         13.046474
std         180.733597
min           1.000000
25%           2.000000
50%           6.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64

In [104]:
quantity_high = np.quantile(df_cleaned.Quantity, 0.99)
print("99%的商品数量控制在(1, {})之间".format(quantity_high))

99%的商品数量控制在(1, 120.0)之间


Note: 由于Quantity最大值为80995，但百分之99的商品数量在1到120之间，难免让人觉得这些量大的订单是极少数情况。而且我们预测的是大多数用户的购买行为，针对这些极少数情况的数据没有什么利用价值。因此把1-120外的数据删除，留下99%的数据。

In [106]:
df_cleaned = df_cleaned.loc[(df_cleaned.Quantity <= 120)]

df_cleaned.shape[0]

392481

In [112]:
# 再看UnitPrice
display(df_cleaned.UnitPrice.describe())
# 发现居然单价最小值有0

UnitPrice_high = np.quantile(df_cleaned.UnitPrice, 0.99)
UnitPrice_low = np.quantile(df_cleaned.UnitPrice, 0.01)
print("98%的数据在", (UnitPrice_low, UnitPrice_high), "中")
# 发现极大值太大

count    392481.000000
mean          2.883001
std           4.280902
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max         649.500000
Name: UnitPrice, dtype: float64

98%的数据在 (0.29, 12.75) 中


In [155]:
price_notzero = df_cleaned[df_cleaned.UnitPrice != 0]["StockCode"].unique()
price_zero = pd.DataFrame(df_cleaned[df_cleaned.UnitPrice == 0]["StockCode"])
price_zero["NewOrNot"] = price_zero["StockCode"].apply(lambda l: l not in price_notzero)
display(price_zero.T)
# 发现单价为0的商品并不是第一次出现，是否可以借鉴同商品的单价？

,9302,33576,40089,47068,47070,56674,86789,130188,139453,145208,...,420404,436597,439361,446125,446793,446794,454463,454464,479079,480649
StockCode,22841,22580,22423,22090,22553,22168,84535B,22062,22055,22162,...,21208,22065,22385,22625,22899,23480,22437,22089,22464,23407
NewOrNot,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [156]:
print(df_cleaned[df_cleaned.StockCode == "22841"]["UnitPrice"].value_counts())
# 发现同商品有多个价格

7.95     187
6.95       9
15.79      2
5.49       2
0.00       1
Name: UnitPrice, dtype: int64


Note: 通过分析单价我们发现，单价为0的商品有其他多个价格，因此我们不能直接替换成其他任意价格，或许为0的价格是因为作为附加的礼品赠送的，因此这不是我们盈利的来源，我们把这些数据也删除。

In [172]:
high_price_commerce = df_cleaned[df_cleaned.UnitPrice > 150].StockCode.unique()
display(high_price_commerce)

array(['22827', '22655', '22828', '22826', '22656', '22502'], dtype=object)

In [182]:
display(df_cleaned[df_cleaned["StockCode"].map(lambda l: l in high_price_commerce)].groupby(["StockCode", "UnitPrice"])["Quantity"].agg(["sum", "count"]))

display(df_cleaned[(df_cleaned["StockCode"] == "22502") & (df_cleaned["UnitPrice"] - 649.50 == 0)])

sum  count
StockCode UnitPrice            
22502     2.00         1      1
          4.95       740     29
          5.95       686    207
          649.50      61      2
22655     50.00        3      2
          125.00      52     31
          295.00       5      5
22656     50.00        1      1
          125.00      22     16
          295.00       3      3
22826     85.00       46     30
          175.00       2      1
          195.00      10     10
22827     145.00      18      9
          165.00      17     17
22828     145.00       6      3
          165.00       4      4

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.5,15098,United Kingdom
222682,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.5,15098,United Kingdom


Note: 看似除了StockCode为22502的商品其他商品单价都很合理。商品22502的单价649.50和它其他的单价相差大，这里这两个订单信息也属于极少数情况，对我们预测大多数用户购买行为没有什么帮助，因此删去。

In [183]:
df_cleaned = df_cleaned.loc[df_cleaned["UnitPrice"] < 500]

In [184]:
df_cleaned.shape[0]

392479

In [186]:
# 对于InvoiceDate
df_cleaned["InvoiceDate"].describe()
# 挺合理的，时间的跨度是12个月

c:\users\edwin\anaconda3\envs\e-commerce data\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until


count                  392479
unique                  16833
top       2011-11-14 15:27:00
freq                      541
first     2010-12-01 08:26:00
last      2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

In [223]:
# 对于country
country_info = pd.DataFrame(df_cleaned.Country.value_counts()).rename(columns={"Country": "Total"})
country_info["Rate"] = (country_info["Total"].map(lambda l: round(l/ df_cleaned.shape[0] * 100, 2)))

display(country_info.sort_values(by="Rate", ascending=False).iloc[0:10])

,Total,Rate
United Kingdom,351257,89.50
Germany,8618,2.20
France,8005,2.04
EIRE,6956,1.77
Spain,2404,0.61
Belgium,1932,0.49
Netherlands,1840,0.47
Switzerland,1808,0.46
Portugal,1425,0.36
Norway,1043,0.27


Note: 可以看到接近90%的数据来自英国。

In [ ]:
# 最后我们查看Description
df_cleaned


